In [1]:
! python3 --version

Python 3.12.3


In [ ]:
! pip install transformers torch accelerate

In [ ]:
! pip install python_dotenv

In [ ]:
! pip install numba

In [ ]:
! pip freeze > requirements.txt

### Huggingface login

In [ ]:
# manual
# Do this from terminal so you can paste the token when requested
! huggingface-cli login

In [5]:
# Programmatically
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

access_token_read = os.environ.get('HUGGINGFACE_TOKEN')

login(token = access_token_read)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/vrbabu/.cache/huggingface/token
Login successful


In [3]:
# Logout
from huggingface_hub import logout
logout()

Successfully logged out.


/Users/vrbabu/Library/CloudStorage/OneDrive-Personal/Documents/Documents/code/Llama2_with_Hugging_Face_Pipeline/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
! huggingface-cli whoami

Gilgamesh37


### Load Model & Tokenizer

In [11]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model, token=True)

### Set up pipeline

In [12]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto"
)

/Users/vrbabu/Library/CloudStorage/OneDrive-Personal/Documents/Documents/code/Llama2_with_Hugging_Face_Pipeline/venv/lib/python3.12/site-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 52432000 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.64s/it]


### Getting responses

In [17]:
from numba import jit, cuda

# @jit(target_backend='cuda') 
from numba import njit
@njit
def get_llama_response(prompt: str)-> None:
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256
    )

    print(f"Chatbot: {sequences[0]['generated_text']}")

### Questions

In [18]:
get_llama_response("Which Goddess has her temple in Ammankoil, Padavedu in India?")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'llama_pipeline': Cannot determine Numba type of <class 'transformers.pipelines.text_generation.TextGenerationPipeline'>

File "../../../../../../../../../var/folders/nc/9qhrjynn6vjfcdgr9x5llf180000gn/T/ipykernel_93479/2506541732.py", line 5:
<source missing, REPL/exec in use?>
